In [1]:
import math
import matplotlib
import json
import requests
import time
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.dates as dates

from datetime import date, datetime, time, timedelta
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook

%matplotlib inline

In [2]:
pd.options.mode.chained_assignment = None # turn off warning message

In [3]:
import misc_function2 as mf
#request api of current daily stock of selected symbol
company_name = 'AAPL'
funct = "history"
token = "YFl1bfK9zAddFzcsOgenstMkFg3Vl84lb6rtf80vAbQ2yj1ldqM4lv75cE5q"
fromdt = "2015-01-01"
url = (f"https://api.worldtradingdata.com/api/v1/history?symbol={company_name}&date_from={fromdt}&sort=newest&api_token={token}")
print(url)

https://api.worldtradingdata.com/api/v1/history?symbol=AAPL&date_from=2015-01-01&sort=newest&api_token=YFl1bfK9zAddFzcsOgenstMkFg3Vl84lb6rtf80vAbQ2yj1ldqM4lv75cE5q


In [4]:
#convert response to python json list
data = requests.get(url).json()[funct]

In [5]:
df = pd.DataFrame(data)
df = df.T

In [6]:
df.tail()

,open,close,high,low,volume
2015-01-08,109.23,111.89,112.15,108.70,59364547
2015-01-07,107.20,107.75,108.20,106.70,40105934
2015-01-06,106.54,106.26,107.43,104.63,65797116
2015-01-05,108.29,106.25,108.65,105.41,64285491
2015-01-02,111.39,109.33,111.44,107.35,53204626


In [7]:
# Rename column name with capitals

df.columns = ['Open','High','Low','Close','Volume']

In [8]:
df = df[['Open','High','Low','Close']].stack().astype('float').unstack()

In [9]:
# Convert Date column to datetime
df.index = pd.to_datetime(df.index,format='%Y-%m-%d')

In [10]:
df = df.sort_index()

In [11]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

Using backtesting library from https://pypi.org/project/Backtesting/

In [12]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 15
    n2 = 60
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            self.position.close()
        
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10000, commission=.002)
bt.run()

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Start                     2015-01-02 00:00:00
End                       2019-11-01 00:00:00
Duration                   1764 days 00:00:00
Exposure [%]                          50.2268
Equity Final [$]                      14615.1
Equity Peak [$]                       14615.1
Return [%]                            46.1506
Buy & Hold Return [%]                 132.101
Max. Drawdown [%]                    -24.0338
Avg. Drawdown [%]                    -4.50758
Max. Drawdown Duration      566 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                   10
Win Rate [%]                               50
Best Trade [%]                        25.8061
Worst Trade [%]                      -9.79097
Avg. Trade [%]                        2.18697
Max. Trade Duration         187 days 00:00:00
Avg. Trade Duration          89 days 00:00:00
Expectancy [%]                        8.89181
SQN                                   0.51867
Sharpe Ratio                      

In [13]:
bt.plot()

### 